In [2]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
!conda install -c conda-forge beautifulsoup4 --yes
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Block1: Success')

Solving environment: ...working... done

# All requested packages already installed.

Block1: Success


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [3]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Block2: Success')

Block2: Success


In [4]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Block3: Success')

Block3: Success


In [5]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
!pip install folium

In [7]:
# define the dataframe columns
column_names = ['City', 'Latitude', 'Longitude'] 

# instantiate the dataframe
LittleElmBase = pd.DataFrame(columns=column_names)
LittleElmBase

,City,Latitude,Longitude


In [8]:
address = 'Little ELm, Tx'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Little Elm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Little Elm are 33.1626194, -96.9375051.


In [9]:
LittleElmBase = LittleElmBase.append({'City': "Little Elm",
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)
LittleElmBase

,City,Latitude,Longitude
0,Little Elm,33.162619,-96.937505


In [19]:
map_LE = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, city in zip(LittleElmBase['Latitude'], LittleElmBase['Longitude'], LittleElmBase['City']):
    label = '{}, {}'.format(LittleElmBase, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=475,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
                parse_html=False).add_to(map_LE)  
    
map_LE

In [20]:
CLIENT_ID = 'T1OZISKZBTCE0XFFU33KFTNG5URO0UDPK4NHM51DXQCAXFR5' # your Foursquare ID
CLIENT_SECRET = 'ZQDGLQHM4ZPO40MICTBCCTMDGPQE24R5MHRUCIOBYEQ23FVO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T1OZISKZBTCE0XFFU33KFTNG5URO0UDPK4NHM51DXQCAXFR5
CLIENT_SECRET:ZQDGLQHM4ZPO40MICTBCCTMDGPQE24R5MHRUCIOBYEQ23FVO


In [21]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 15000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=T1OZISKZBTCE0XFFU33KFTNG5URO0UDPK4NHM51DXQCAXFR5&client_secret=ZQDGLQHM4ZPO40MICTBCCTMDGPQE24R5MHRUCIOBYEQ23FVO&v=20180605&ll=33.1626194,-96.9375051&radius=15000&limit=200'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e99949614a126001ba961d1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Little Elm',
  'headerFullLocation': 'Little Elm',
  'headerLocationGranularity': 'city',
  'totalResults': 200,
  'suggestedBounds': {'ne': {'lat': 33.29761953500013,
    'lng': -96.77653910563417},
   'sw': {'lat': 33.02761926499986, 'lng': -97.09847109436582}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '554cf6da498eb7864c57187b',
       'name': 'Little Elm Crafthouse',
       'location': {'address': '850 W. Eldorado Pkwy Ste. 700',
        'lat': 33.155466954821726,
        'lng': -96.94499925538383,
        'labeledLatLngs

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Little Elm Crafthouse,Bar,33.155467,-96.944999
1,Beard Park,Park,33.163368,-96.932129
2,Little Elm Park,Park,33.159002,-96.947125
3,Beach,Beach,33.159321,-96.947473
4,Little Elm Rec Center,Athletics & Sports,33.165091,-96.939876


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [28]:
Frisco_address = 'Frisco, Tx'

geolocator = Nominatim(user_agent="my-application")
Frisco_location = geolocator.geocode(Frisco_address)
Frisco_latitude = Frisco_location.latitude
Frisco_longitude = Frisco_location.longitude
print('The geograpical coordinate of Frisco are {}, {}.'.format(Frisco_latitude,Frisco_longitude))

The geograpical coordinate of Frisco are 33.1506744, -96.8236116.


In [29]:
# define the dataframe columns
Frisco_column_names = ['City', 'Latitude', 'Longitude'] 

# instantiate the dataframe
Frisco_Base = pd.DataFrame(columns=Frisco_column_names)
Frisco_Base = Frisco_Base.append({'City': "Frisco",
                                          'Latitude': Frisco_latitude,
                                          'Longitude': Frisco_longitude}, ignore_index=True)
Frisco_Base

,City,Latitude,Longitude
0,Frisco,33.150674,-96.823612


In [30]:
# create map using latitude and longitude values
map_Frisco = folium.Map(location=[Frisco_latitude, Frisco_longitude], zoom_start=13)

# add markers to map
for lat, lng, city in zip(Frisco_Base['Latitude'], Frisco_Base['Longitude'], Frisco_Base['City']):
    label = '{}, {}'.format(Frisco_Base, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=475,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
                parse_html=False).add_to(map_Frisco)  
    
map_Frisco

In [31]:
# type your answer here
LIMIT1 = 200 # limit of number of venues returned by Foursquare API
radius1 = 15000 # define radius
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Frisco_latitude, 
    Frisco_longitude, 
    radius1, 
    LIMIT1)
url1 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=T1OZISKZBTCE0XFFU33KFTNG5URO0UDPK4NHM51DXQCAXFR5&client_secret=ZQDGLQHM4ZPO40MICTBCCTMDGPQE24R5MHRUCIOBYEQ23FVO&v=20180605&ll=33.1506744,-96.8236116&radius=15000&limit=200'

In [32]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5e9995a0a2e538001b63c899'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Frisco',
  'headerFullLocation': 'Frisco',
  'headerLocationGranularity': 'city',
  'totalResults': 229,
  'suggestedBounds': {'ne': {'lat': 33.28567453500013,
    'lng': -96.66266752771442},
   'sw': {'lat': 33.015674264999866, 'lng': -96.9845556722856}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6d2bd4f964a520a3d21fe3',
       'name': "Babe's Chicken Dinner House",
       'location': {'address': '6475 Page Rd',
        'crossStreet': 'at John W Elliott Dr',
        'lat': 33.14903143006786,
        'lng': -96.830446393129

In [33]:
# function that extracts the category of the venue
def get_Frisco_category_type(row):
    try:
        Frisco_categories_list = row['categories']
    except:
        Frisco_categories_list = row['venue.categories']
        
    if len(Frisco_categories_list) == 0:
        return None
    else:
        return Frisco_categories_list[0]['name']

In [34]:
Frisco_venues = results1['response']['groups'][0]['items']
    
Frisco_nearby_venues = json_normalize(Frisco_venues) # flatten JSON

# filter columns
Frisco_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Frisco_nearby_venues =Frisco_nearby_venues.loc[:, Frisco_filtered_columns]

# filter the category for each row
Frisco_nearby_venues['venue.categories'] = Frisco_nearby_venues.apply(get_Frisco_category_type, axis=1)

# clean columns
Frisco_nearby_venues.columns = [col.split(".")[-1] for col in Frisco_nearby_venues.columns]

Frisco_nearby_venues.head()

,name,categories,lat,lng
0,Babe's Chicken Dinner House,Southern / Soul Food Restaurant,33.149031,-96.830446
1,Mariana's Taco Shop,Mexican Restaurant,33.151367,-96.824001
2,Frisco Public Library,Library,33.150182,-96.834209
3,Frisco Commons,Park,33.153426,-96.813934
4,"ILoveKickboxing.com - Frisco, TX",Gym / Fitness Center,33.149256,-96.805984


In [35]:
print('{} venues were returned by Foursquare.'.format(Frisco_nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [36]:
#sorted by the count in decending order to get top 15
LE_grouped = nearby_venues.groupby('categories').count().reset_index()
LE_grouped.sort_values(by=['name'],inplace=True, ascending=False)
LE_grouped.head(15)

,categories,name,lat,lng
35,Mexican Restaurant,9,9,9
15,Fast Food Restaurant,7,7,7
41,Park,6,6,6
31,Italian Restaurant,6,6,6
13,Coffee Shop,4,4,4
25,Grocery Store,4,4,4
27,Gym / Fitness Center,3,3,3
34,Liquor Store,3,3,3
22,Golf Course,3,3,3
42,Pizza Place,3,3,3


In [37]:
LE_grouped = nearby_venues.groupby('categories').count().reset_index()
LE_grouped.sort_values(by=['name'],inplace=True)
LE_grouped.head(10)

,categories,name,lat,lng
0,American Restaurant,1,1,1
23,Golf Driving Range,1,1,1
24,Greek Restaurant,1,1,1
50,Warehouse Store,1,1,1
28,Hardware Store,1,1,1
29,Hotel,1,1,1
32,Library,1,1,1
33,Lingerie Store,1,1,1
36,Motorcycle Shop,1,1,1
37,Movie Theater,1,1,1


In [38]:
#sorted by the count in decending order to get top 15
Frisco_grouped = Frisco_nearby_venues.groupby('categories').count().reset_index()
Frisco_grouped.sort_values(by=['name'],inplace=True,ascending=False)
Frisco_grouped.head(15)

,categories,name,lat,lng
38,Mexican Restaurant,6,6,6
30,Ice Cream Shop,5,5,5
17,Fast Food Restaurant,5,5,5
31,Italian Restaurant,5,5,5
12,Coffee Shop,5,5,5
54,Sushi Restaurant,4,4,4
27,Gym / Fitness Center,3,3,3
24,Grocery Store,3,3,3
53,Supermarket,2,2,2
49,Southern / Soul Food Restaurant,2,2,2


In [39]:
Frisco_grouped = Frisco_nearby_venues.groupby('categories').count().reset_index()
Frisco_grouped.sort_values(by=['name'],inplace=True)
Frisco_grouped.head(10)

,categories,name,lat,lng
59,Wine Shop,1,1,1
23,Greek Restaurant,1,1,1
48,Shopping Plaza,1,1,1
25,Gun Range,1,1,1
47,Shopping Mall,1,1,1
46,Pub,1,1,1
28,Hardware Store,1,1,1
58,Warehouse Store,1,1,1
45,Pizza Place,1,1,1
32,Japanese Restaurant,1,1,1
